In [1]:
import os
os.environ["NGSIM_DIR"] = "/home/surender/Downloads/NGSIM"
os.environ["OPENDD_DIR"] = "/home/surender/Downloads/openDD"
os.environ["CARLA_PATH"] = "/home/surender/Downloads/carlaOld"
import sys
#sys.path.append('/home/surender/Downloads/CARLA_0.9.9.4/PythonAPI/carla/dist')
import carla
import random
import argparse

from carla_real_traffic_scenarios.carla_maps import CarlaMaps
from carla_real_traffic_scenarios.ngsim import NGSimDatasets, DatasetMode
from carla_real_traffic_scenarios.ngsim.scenario import NGSimLaneChangeScenario
from carla_real_traffic_scenarios.opendd.scenario import OpenDDScenario
from carla_real_traffic_scenarios.reward import RewardType
from carla_real_traffic_scenarios.scenario import Scenario

from carla_birdeye_view import BirdViewProducer, BirdViewCropType, PixelDimensions
from PIL import Image
from IPython.display import clear_output, Image, display, HTML
import cv2

%matplotlib tk
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import threading
import time



In [ ]:
from src.VehicleActor import *

In [ ]:
def cmd_carla1():
    os.system("/home/surender/Downloads/carlaOld/CarlaUE4.sh -benchmark -fps=10 -quality-level=Low -opengl -Resx=300 -Resy=300 -NoVSync")

In [2]:
def cmd_carla():
    os.system("DISPLAY= /home/surender/Downloads/carlaOld/CarlaUE4.sh -benchmark -fps=10 -quality-level=Low -opengl -Resx=300 -Resy=300 -NoVSync")

In [3]:
'''
def parser_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--dataset", choices=["ngsim", "opendd"], default="opendd")
    parser.add_argument("--host", default="127.0.0.1")
    parser.add_argument("--port", default=2000, type=int)
    parser.add_argument("--num-episodes", default=10, type=int)
    args = parser.parse_args()
    return args

'''

def prepare_ngsim_scenario(client: carla.Client) -> Scenario:
    data_dir = os.environ.get("NGSIM_DIR")
    #data_dir = os.listdir('/home/surender/Downloads/NGSIM')
    assert data_dir, "Path to the directory with NGSIM dataset is required"
    ngsim_map = NGSimDatasets.list()
    ngsim_dataset = ngsim_map[1]
    client.load_world(ngsim_dataset.carla_map.level_path)
    return NGSimLaneChangeScenario(
        ngsim_dataset,
        dataset_mode=DatasetMode.TRAIN,
        data_dir=data_dir,
        reward_type=RewardType.DENSE,
        client=client,
    )

'''
def prepare_opendd_scenario(client: carla.Client) -> Scenario:
    data_dir = os.environ.get("OPENDD_DIR")
    assert data_dir, "Path to the directory with openDD dataset is required"
    maps = ["rdb1", "rdb2", "rdb3", "rdb4", "rdb5", "rdb6", "rdb7"]
    map_name = random.choice(maps)
    carla_map = getattr(CarlaMaps, map_name.upper())
    client.load_world(carla_map.level_path)
    return OpenDDScenario(
        client,
        dataset_dir=data_dir,
        dataset_mode=DatasetMode.TRAIN,
        reward_type=RewardType.DENSE,
        place_name=map_name,
    )

'''
def prepare_ego_vehicle(world: carla.World) -> carla.Actor:
    car_blueprint = world.get_blueprint_library().find("vehicle.audi.a2")

    # This will allow external scripts like manual_control.py or no_rendering_mode.py
    # from the official CARLA examples to take control over the ego agent
    car_blueprint.set_attribute("role_name", "hero")

    # spawn points doesnt matter - scenario sets up position in reset
    ego_vehicle = world.spawn_actor(
        car_blueprint, carla.Transform(carla.Location(0, 0, 500), carla.Rotation())
    )

    assert ego_vehicle is not None, "Ego vehicle could not be spawned"

    # Setup any car sensors you like, collect observations and then use them as input to your model
    return ego_vehicle

In [4]:
p = threading.Thread(target = cmd_carla)
p.start()


In [5]:
host = "localhost"
port = 2000
client = carla.Client(host,port)

In [6]:
scenario = prepare_ngsim_scenario(client)

Switch synchronous_mode=True
Change fixed_delta_seconds=0.1


In [7]:
world = client.get_world()
spectator = world.get_spectator()
ego_vehicle = prepare_ego_vehicle(world)




In [18]:
birdview_producer = BirdViewProducer(
    client,  # carla.Client
    target_size=PixelDimensions(width=150, height=336),
    pixels_per_meter=4,
    crop_type=BirdViewCropType.FRONT_AND_REAR_AREA
    )

Cache file does not exist, generating cache at birdview_v2_cache/US-101__px_per_meter=4__opendrive_hash=d3f99b7fd2bbc1edeccfc6adf7734070cfa49743__margin=300.npy


In [21]:
world.tick()

68976162

In [339]:
done = False
while not done:
    ego_vehicle.apply_control(carla.VehicleControl(throttle=1, steer=0))
    world.tick()
    a,b,c,d = scenario.step(ego_vehicle)
    print(b,a.name)
    done = c
    birdview = birdview_producer.produce(
            agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
            )
    rgb = BirdViewProducer.as_rgb(birdview)
    cv2.imshow('Frame',rgb)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

0.0 CHANGE_LANE_RIGHT
-1 LANE_FOLLOW


In [338]:
scenario.reset(ego_vehicle)

In [329]:
ego_vehicle.apply_control(carla.VehicleControl(throttle=1, steer=0))

In [330]:
world.tick()

68976203

In [331]:
a,b,c,d = scenario.step(ego_vehicle)

In [332]:
b

0.0

In [333]:
a.name

'LANE_FOLLOW'

In [334]:
a.value

0

In [335]:
d

{'ngsim_dataset': {'road': 'US101',
  'timeslice': '0820am-0835am',
  'frame': 4378,
  'dataset_mode': 'TRAIN'},
 'scenario_data': {'ego_veh': <carla.libcarla.Vehicle at 0x7f445371c750>,
  'original_veh_transform': <carla.libcarla.Transform at 0x7f43783e8af0>,
  'original_to_ego_distance': 18.436887741088867},
 'reward_type': 'DENSE',
 'on_start_lane': False,
 'on_target_lane': True,
 'is_junction': True,
 'alignment_errors': 'track=1.39m yaw=0.06rad',
 'target_alignment_counter': 0}

In [336]:
birdview = birdview_producer.produce(
            agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
            )
rgb = BirdViewProducer.as_rgb(birdview)
cv2.imshow('Frame',rgb)
if cv2.waitKey(25) & 0xFF == ord('q'):
    cv2.destroyAllWindows()
      

In [ ]:
manager = VehicleManager(ego_vehicle,world,scenario)

In [ ]:
world.tick()

In [ ]:
manager.update_neighbours()

In [ ]:
manager.neighbour_profiles

In [ ]:
manager.update_buffers()

In [ ]:
e,n = manager.return_data()

In [ ]:
world.tick()


In [ ]:
manager.vehicleSensorslist[1764].current_frame

In [ ]:
plt.imshow(e[0])

In [ ]:
plt.imshow(e[1])

In [ ]:
plt.imshow(n[0][1])

In [ ]:
manager.update_neighbours()
world.tick()
time.sleep(1)
manager.update_buffers()
ego,n = manager.return_data()

In [ ]:
manager.update_buffers()

In [ ]:
ego,n = manager.return_data()

In [ ]:
ego

In [ ]:
n

In [ ]:
sys.getsizeof(n)

In [ ]:
n[0][0].shape

In [ ]:
sys.getsizeof(e)

In [ ]:
manager.vehicleSensorslist[0].start_sensors()

In [ ]:
manager.vehicleSensorslist[0].gnss_buffer

In [ ]:
scenario.step(ego_vehicle).chauffeur_cmd = 1

In [10]:
scenario._lane_change

LaneChangeInstant(timeslot=NGSimTimeslot(file_suffix='0750am-0805am', blacklisted_vehicle_ids=frozenset({2851, 2691, 2820, 2809, 2871, 2873})), frame_start=6177, vehicle_id=2056, lane_from=4, lane_to=3)

In [ ]:
vehicles[0]._vehicle_by_vehicle_id

In [ ]:
scenario._

In [13]:
ego_vehicle.get_transform().location.z

500.0

In [11]:
l1 = list(scenario._ngsim_vehicles_in_carla._vehicle_by_vehicle_id.keys())

In [12]:
l1

[2048,
 2050,
 2052,
 2053,
 2054,
 2055,
 2057,
 2059,
 2060,
 2061,
 2065,
 2067,
 2068,
 2069,
 2071,
 2072,
 2073,
 2074,
 2079,
 2080,
 2082,
 2084,
 2085,
 2086,
 2088,
 2089,
 2090,
 2092,
 2093,
 2096,
 2098,
 2099,
 2102,
 2103,
 2104,
 2105,
 2106,
 2107,
 2108,
 2111,
 2112,
 2115,
 2116,
 2118,
 2119,
 2124,
 2125,
 2126,
 2127,
 2128,
 2129,
 2130,
 2131,
 2132,
 2133,
 2134,
 2135,
 2136,
 2138,
 2139,
 2140,
 2142,
 2144,
 2148,
 2150,
 2156,
 2162,
 2170,
 1931,
 1937,
 1945,
 1955,
 1961,
 1964,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1987,
 1988,
 1989,
 1991,
 1992,
 1995,
 1996,
 1998,
 1999,
 2000,
 2001,
 2003,
 2006,
 2007,
 2008,
 2010,
 2015,
 2016,
 2019,
 2031,
 2032,
 2033,
 2034,
 2035,
 2036,
 2037,
 2039,
 2041,
 2042,
 2043,
 2044,
 2045,
 2046]

In [14]:
vehicles = scenario._ngsim_recording.step()

In [15]:
vehicles

[RealTrafficVehicle(id=2048, type_id='vehicle.mustang.mustang', timestamp_s=0.2, width_m=1.7983200000000001, length_m=5.1816, transform=Transform(position=Vector3(x=-10.780916665247284, y=0.26985530487004145, z=-0.007357253693044186), orientation=Vector2(x=0.9983903761296602, y=-0.056715578562470094)), speed=82.17191637135615, debug=False),
 RealTrafficVehicle(id=2050, type_id='vehicle.audi.etron', timestamp_s=0.2, width_m=1.9507200000000002, length_m=5.0292, transform=Transform(position=Vector3(x=-38.04565192499647, y=4.593618098231916, z=-0.050091702491045), orientation=Vector2(x=0.999998833902921, y=0.001527151858282175)), speed=83.71893697531488, debug=False),
 RealTrafficVehicle(id=2052, type_id='vehicle.lincoln.mkz2017', timestamp_s=0.2, width_m=2.10312, length_m=4.8768, transform=Transform(position=Vector3(x=-51.371305269511915, y=4.795714377180053, z=-0.011359786614775658), orientation=Vector2(x=0.9999988462982949, y=0.001519013521693143)), speed=85.03567345556952, debug=False)

In [ ]:
l3 = [x.id for x in vehicles]

In [ ]:
scenario

In [ ]:
len(d.keys())

In [ ]:
len(l3)

In [21]:
world.tick()

1569

In [ ]:
#scenario.step(ego_vehicle)

In [22]:
d = scenario._ngsim_vehicles_in_carla._vehicle_by_vehicle_id

In [23]:
dist = {}
for i in d:
    c = ego_vehicle.get_location().distance(d[i].get_location())
    dist[i] = c

In [24]:
updated_dist = dict(sorted(dist.items(), key=lambda item: item[1]))

In [25]:
list(updated_dist.keys())[:4]

[2052, 2053, 2054, 2050]

In [26]:
updated_dist

{2052: 5.052845478057861,
 2053: 6.285128593444824,
 2054: 7.057491779327393,
 2050: 11.527693748474121,
 2055: 21.442785263061523,
 2042: 25.643177032470703,
 2057: 25.651817321777344,
 2046: 26.652050018310547,
 2045: 27.87602424621582,
 2041: 28.615638732910156,
 2069: 29.317975997924805,
 2061: 31.360370635986328,
 2060: 32.813575744628906,
 2044: 33.462398529052734,
 2059: 37.85524368286133,
 2048: 38.06349563598633,
 2065: 41.93913650512695,
 2067: 43.2357292175293,
 2068: 43.436859130859375,
 2039: 45.41132354736328,
 2037: 46.29105758666992,
 2071: 49.79523849487305,
 2033: 51.01645278930664,
 2043: 52.65338134765625,
 2073: 55.018184661865234,
 2032: 56.49943542480469,
 2074: 57.239009857177734,
 2019: 57.55589294433594,
 2072: 57.64030456542969,
 2034: 61.78107833862305,
 2082: 64.46503448486328,
 2084: 66.14724731445312,
 2079: 69.08745574951172,
 2086: 72.88870239257812,
 2035: 74.93132781982422,
 2080: 77.68678283691406,
 2085: 78.8862075805664,
 2036: 80.59092712402344,
 

In [29]:
 kip = dict((k, v) for k, v in updated_dist.items() if v <= 10)

In [30]:
kip

{2052: 5.052845478057861, 2053: 6.285128593444824, 2054: 7.057491779327393}

In [35]:
d[2052].get_location().x,d[2052].get_location().y,d[2052].get_location().z

(-52.67055130004883, 4.793801784515381, -0.011367186903953552)

In [42]:
ego_vehicle.get_location().x,ego_vehicle.get_location().y,ego_vehicle.get_location().z

(-50.10076904296875, 0.44323238730430603, -0.0073437499813735485)

In [43]:
d[2053].get_location().x,d[2053].get_location().y,d[2053].get_location().z

(-52.03022384643555, -5.53840970993042, -0.0073437499813735485)

In [44]:
d[2054].get_location().x,d[2054].get_location().y,d[2054].get_location().z

(-56.53384780883789, -2.458782911300659, 0.03734374791383743)

In [57]:
limit = 40

In [58]:
dist3 = {}
for i in d:
    if d[i].get_location().x <= ego_vehicle.get_location().x + limit and d[i].get_location().x >= ego_vehicle.get_location().x:
        vector_direction = (d[i].get_location().x - ego_vehicle.get_location().x,d[i].get_location().y - ego_vehicle.get_location().y,d[i].get_location().z - ego_vehicle.get_location().z)
        dist3[i] = [ego_vehicle.get_location().distance(d[i].get_location()),vector_direction]

In [59]:
dist3

{2048: [38.06349563598633, (38.06339359283447, -0.08852514624595642, 0.0)],
 2050: [11.527693748474121,
  (10.755302429199219, 4.148418098688126, -0.042734373826533556)],
 2041: [28.615638732910156,
  (27.105459213256836, -9.173266261816025, -0.0042968750931322575)],
 2042: [25.643177032470703,
  (25.55194854736328, -2.15964612364769, 0.08011718420311809)],
 2044: [33.462398529052734,
  (33.18251419067383, 4.318160206079483, 0.08011718420311809)],
 2045: [27.87602424621582, (27.86549186706543, 0.7661674320697784, 0.0)],
 2046: [26.652050018310547, (26.066490173339844, -5.556062549352646, 0.0)]}

In [65]:
avg_throttle = 0
avg_steer = 0
for i in dist3:
    avg_throttle += dist3[i][1][0]
    avg_steer += dist3[i][1][1]
avg_throttle = avg_throttle/len(dist3.keys())
avg_steer *= -1

In [66]:
avg_throttle

26.94151428767613

In [67]:
avg_steer

7.74475434422493

In [53]:
ego_vehicle.get_location().x

-50.10076904296875

In [52]:
for i in d:
    print(d[i].get_location().x)

-12.037375450134277
-39.34546661376953
-52.67055130004883
-52.03022384643555
-56.53384780883789
-69.43990325927734
-74.92628479003906
-86.8130874633789
-82.7774658203125
-81.44981384277344
-91.60293579101562
-93.3343505859375
-93.18397521972656
-79.15559387207031
-99.89262390136719
-107.70559692382812
-104.15877532958984
-106.95130920410156
-118.42607879638672
-127.40078735351562
-114.49571990966797
-116.13971710205078
-128.75149536132812
-122.98944854736328
-132.24571228027344
-137.97032165527344
-140.17066955566406
-140.740234375
-134.61410522460938
-156.82888793945312
-157.58412170410156
-189.88401794433594
-155.7015838623047
-148.94961547851562
-184.16966247558594
-160.96627807617188
-175.1715850830078
-208.48304748535156
-179.9329833984375
-175.72157287597656
-176.5337677001953
-198.01028442382812
-225.58587646484375
-187.54306030273438
-194.30702209472656
-212.0138397216797
-247.1220245361328
-205.05975341796875
-228.0970458984375
-265.7156066894531
-234.29286193847656
-199.04565

In [ ]:
dista

In [ ]:
d

In [ ]:
d

In [ ]:
d[ego_vehicle.id]

In [ ]:
world.tick()

In [ ]:
ego_vehicle.get_location().distance(d[48].get_location())

In [ ]:
min_d = 10000
min_ind = -1
for i in l1:
    dist = ego_vehicle.get_location().distance(d[i].get_location())
    print(dist)
    if dist<min_d:
        min_d = dist
        min_ind = i

In [ ]:
min_ind

In [ ]:
min_d

In [ ]:
d[146].get_location().z

In [ ]:
ego_vehicle.get_location().z

In [ ]:
d[48].get_location().z

In [ ]:
(d[48].get_location() + ego_vehicle.get_location()).z

In [ ]:
d[236].get_location().x

In [ ]:
l1

In [ ]:
list(world.get_actors())

In [ ]:
ego_vehicle.get_location().z

In [ ]:
c  =vehicles[0].transform.position

In [ ]:
c.x

In [ ]:
ego_vehicle.id

In [ ]:
len(l3)

In [ ]:
l1

In [ ]:
scenario.reset(ego_vehicle)

In [ ]:
for i in range(len(l1)):
    if l1[i] == l3[i]:
        continue
    else:
        print(l1[i],l3[i],i)

In [ ]:
vehicles[0].transform == ego_vehicle.get_transform()

In [ ]:
vehicles = scenario._ngsim_recording.step()

In [ ]:

scenario._ngsim_vehicles_in_carla

In [ ]:
world.tick()

In [ ]:
world.get_actors()[1]

In [ ]:
list(world.get_actors())

In [ ]:


birdview_producer = BirdViewProducer(
    client,  # carla.Client
    target_size=PixelDimensions(width=150, height=336),
    pixels_per_meter=4,
    crop_type=BirdViewCropType.FRONT_AND_REAR_AREA
)

In [72]:
world.tick()

1571

In [ ]:
def print_data(dat):
    print(dat.latitude,dat.longitude, dat.altitude)

In [ ]:
def prit_data(dat):
    print(dat.frame)
    points = np.frombuffer(dat.raw_data, dtype=np.dtype('f4'))
    points = np.reshape(points, (len(dat), 4))
    print(points)

In [ ]:
def check_img(img):
    print(img.frame)
    array = np.frombuffer(img.raw_data, dtype=np.dtype("uint8")) 
    array = np.reshape(array, (img.height, img.width, 4)) # RGBA format
    array = array[:, :, :3] #  Take only RGB
    
    
    img = Image.fromarray(array)
    img = img.resize((299, 299), Image.ANTIALIAS)
    input_data = np.array(img)

In [71]:
cam_bp = world.get_blueprint_library().find('sensor.camera.rgb')
cam_bp.set_attribute("image_size_x",str(720))
cam_bp.set_attribute("image_size_y",str(720))
cam_bp.set_attribute("fov",str(100))
cam_location = carla.Location(2,0,1)
cam_rotation = carla.Rotation(0,0,0)
cam_transform = carla.Transform(cam_location,cam_rotation)
ego_cam = world.spawn_actor(cam_bp,cam_transform,attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
rgb_front_listener = ego_cam
rgb_front_listener.listen(lambda image: image.save_to_disk('check.jpg'))

In [ ]:
ldr_bp = world.get_blueprint_library().find('sensor.camera.rgb')
ldr_loc = carla.Location(0,0,0)
ldr_rot = carla.Rotation(0,0,0)
ldr_bp.set_attribute("range",'500.0')


ldr_transform = carla.Transform(ldr_loc,ldr_rot)
ego_ldr = world.spawn_actor(ldr_bp,ldr_transform, attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
ego_ldr.listen(lambda dat: prit_data(dat))

In [ ]:
ldr_transform.location.z

In [ ]:
list(world.get_actors())

In [ ]:
world.tick()

In [ ]:
for i in d:
    print(d[i].id,i)

In [ ]:
del ego_sadr

In [ ]:
ego_sadr

In [ ]:
world.tick()

In [ ]:
sadr_bp = world.get_blueprint_library().find('sensor.other.gnss')
sadr_loc = carla.Location(0,0,0)
sadr_rot = carla.Rotation(0,0,0)
#ldr_bp.set_attribute("range",'10.0')


sadr_transform = carla.Transform(sadr_loc,sadr_rot)
ego_sadr = world.spawn_actor(sadr_bp,sadr_transform, attach_to=d[1036], attachment_type=carla.AttachmentType.Rigid)
ego_sadr.listen(lambda dat: print_data(dat))

In [ ]:
sdr_bp = world.get_blueprint_library().find('sensor.other.gnss')
sdr_loc = carla.Location(0,0,0)
sdr_rot = carla.Rotation(0,0,0)
#ldr_bp.set_attribute("range",'10.0')


sdr_transform = carla.Transform(sdr_loc,sdr_rot)
ego_sdr = world.spawn_actor(sdr_bp,sdr_transform, attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
ego_sdr.listen(lambda dat: print_data(dat))

In [ ]:
world.tick()

In [ ]:
sensor.other.imu

In [ ]:
def reload_carla():
    global host,port,client,scenario, world,spectator,ego_vehicle,birdview_producers,p
    p = threading.Thread(target = cmd_carla)
    p.start()
    print("Restarting carla.....")
    time.sleep(5)
    
    
    print("Connecting to carla...")
    host = "localhost"
    port = 2000
    client = carla.Client(host,port)
    scenario = prepare_ngsim_scenario(client)
    world = client.get_world()
    spectator = world.get_spectator()
    ego_vehicle = prepare_ego_vehicle(world)
    birdview_producer = BirdViewProducer(
    client,  # carla.Client
    target_size=PixelDimensions(width=150, height=336),
    pixels_per_meter=4,
    crop_type=BirdViewCropType.FRONT_AND_REAR_AREA
    )
    ldr_bp = world.get_blueprint_library().find('sensor.other.radar')
    ldr_loc = carla.Location(0,0,0)
    ldr_rot = carla.Rotation(0,0,0)
    ldr_bp.set_attribute("range",'10.0')


    ldr_transform = carla.Transform(ldr_loc,ldr_rot)
    ego_ldr = world.spawn_actor(ldr_bp,ldr_transform, attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
    ego_ldr.listen(lambda dat: load_radar(dat))

In [ ]:
class Hp():
    
    def __init__(self):
        self.nb_steps = 1500
        self.episode_length = 1000
        self.learning_rate = 0.02
        self.nb_directions = 16
        self.nb_best_directions = 16
        assert self.nb_best_directions <= self.nb_directions
        self.noise = 0.03
        self.seed = 10
        #self.env_name = 'HalfCheetahBulletEnv-v0'

# Normalizing the states


In [ ]:
hp = Hp()
np.random.seed(hp.seed)
state_limit = 15


input_state = [0]*state_limit*4
nb_inputs = state_limit * 4
nb_outputs = 2
save_every = 25

render = False

In [ ]:
reward_vector = []

In [ ]:

class Normalizer():
    
    def __init__(self, nb_inputs):
        global hp
        self.n = np.zeros(nb_inputs)
        self.mean = np.zeros(nb_inputs)
        self.mean_diff = np.zeros(nb_inputs)
        self.var = np.zeros(nb_inputs)
    
    def observe(self, x):
        global hp
        self.n += 1.
        last_mean = self.mean.copy()
        
        self.mean += (x - self.mean) / self.n
        
        self.mean_diff += (x - last_mean) * (x - self.mean)
        self.var = (self.mean_diff / self.n).clip(min = 1e-2)
    
    def normalize(self, inputs):
        global hp
        obs_mean = self.mean
        obs_std = np.sqrt(self.var)
        if np.sum(obs_std) == 0:
            print("TILTED")
        return (inputs - obs_mean) / obs_std

# Building the AI

class Policy():
    
    def __init__(self, input_size, output_size):
        
        self.theta = np.zeros((output_size, input_size))
    
    def evaluate(self, input, delta = None, direction = None):
        global hp
        if direction is None:
            return self.theta.dot(input)
        elif direction == "positive":
            return (self.theta + hp.noise*delta).dot(input)
        else:
            return (self.theta - hp.noise*delta).dot(input)
    
    def sample_deltas(self):
        global hp
        return [np.random.randn(*self.theta.shape) for _ in range(hp.nb_directions)]
    
    def update(self, rollouts, sigma_r):
        global hp
        step = np.zeros(self.theta.shape)
        for r_pos, r_neg, d in rollouts:
            step += (r_pos - r_neg) * d
        if np.sum(step) == 0:
            print("STEPS ZERO")
            for r_pos, r_neg, d in rollouts:
                step = np.ones(self.theta.shape) * 0.05 * d
            
        if sigma_r == 0:
            print("NO ZERO")
            sigma_r = 0.01
        self.theta += hp.learning_rate / (hp.nb_best_directions * sigma_r) * step
        
    def save_policy(self,step_num):
        np.savetxt("Policy_" + str(step_num) +".gz", self.theta)
    
    def load_policy(self,step_num):
        self.theta = np.loadtxt("Policy_" + str(step_num) +".gz")

# Exploring the policy on one specific direction and over one episode

def explore(env, normalizer, policy, direction = None, delta = None):
    global ego_vehicle,input_state,render,hp
    env.reset(ego_vehicle)
    world.tick()
    state = input_state
    done = False
    num_plays = 0.
    sum_rewards = 0
    while not done:
        normalizer.observe(state)
        state = normalizer.normalize(state)
        action = policy.evaluate(state, delta, direction)
        action[0] = np.clip(action[0],0.0,1.0) # Throttle
        action[1] = np.clip(action[1],-1.0,1.0) # Steering
        #action[2] = np.clip(action[2],0.0,1.0) #Brake
        
        if render:
            birdview = birdview_producer.produce(
            agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
            )
            rgb = BirdViewProducer.as_rgb(birdview)
            cv2.imshow('Frame',rgb)
            if cv2.waitKey(25) & 0xFF == ord('q'):

                  break
        
        ego_vehicle.apply_control(carla.VehicleControl(throttle=action[0], steer=action[1]))
        cmd, reward, done, _ = env.step(ego_vehicle)
        #reward = np.clip(reward,0,1)
        #reward = max(reward, -1)
        #reward = max(min(reward, 1), -1)
        world.tick()
        sum_rewards += reward
        num_plays += 1
    cv2.destroyAllWindows()
    return sum_rewards


def train(env, policy, normalizer, hp, resume=None, resume_step = 0):
    global render,save_every,reward_vector,scenario
    max_reward =  -1
    if resume == True:
        policy.load_policy(resume_step)
        
    for step in range(resume_step, hp.nb_steps):
        try:
            if step > 1000:
                render = True
            # Initializing the perturbations deltas and the positive/negative rewards
            deltas = policy.sample_deltas()
            positive_rewards = [0] * hp.nb_directions
            negative_rewards = [0] * hp.nb_directions

            # Getting the positive rewards in the positive directions
            for k in range(hp.nb_directions):
                positive_rewards[k] = explore(env, normalizer, policy, direction = "positive", delta = deltas[k])

            # Getting the negative rewards in the negative/opposite directions
            for k in range(hp.nb_directions):
                negative_rewards[k] = explore(env, normalizer, policy, direction = "negative", delta = deltas[k])

            # Gathering all the positive/negative rewards to compute the standard deviation of these rewards
            all_rewards = np.array(positive_rewards + negative_rewards)
            sigma_r = all_rewards.std()

            # Sorting the rollouts by the max(r_pos, r_neg) and selecting the best directions
            scores = {k:max(r_pos, r_neg) for k,(r_pos,r_neg) in enumerate(zip(positive_rewards, negative_rewards))}
            order = sorted(scores.keys(), key = lambda x:scores[x], reverse = True)[:hp.nb_best_directions]
            rollouts = [(positive_rewards[k], negative_rewards[k], deltas[k]) for k in order]

            # Updating our policy
            policy.update(rollouts, sigma_r)

            if step%save_every == 0:
                policy.save_policy(step)
            

            # Printing the final reward of the policy after the update
            reward_evaluation = explore(env, normalizer, policy)
            
            if reward_evaluation > max_reward:
                policy.save_policy(step)
                print("Max reward:",reward_evaluation)
                max_reward = reward_evaluation
            print('Step:', step, 'Reward:', reward_evaluation)
            reward_vector.append(reward_evaluation)
        except Exception as e:
            print(e)
            reload_carla()
            time.sleep(5)
            reward_vector.append(reward_vector[-1])
            env = scenario
            print("Carla Reloaded and connected, continue training!")
            continue

In [ ]:
def load_radar(dat):
    global input_state,state_limit
    points = np.frombuffer(dat.raw_data, dtype=np.dtype('f4'))
    points = np.reshape(points, (len(dat), 4))
    radar_Data = points[np.argsort(points[:, -1])]
        
    state = radar_Data[:state_limit].flatten()
    state.resize(state_limit*4,refcheck = False)
    input_state =  state

In [ ]:
ldr_bp = world.get_blueprint_library().find('sensor.other.radar')
ldr_loc = carla.Location(0,0,0)
ldr_rot = carla.Rotation(0,0,0)
ldr_bp.set_attribute("range",'10.0')


ldr_transform = carla.Transform(ldr_loc,ldr_rot)
ego_ldr = world.spawn_actor(ldr_bp,ldr_transform, attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
ego_ldr.listen(lambda dat: load_radar(dat))

In [ ]:
policy = Policy(nb_inputs, nb_outputs)
normalizer = Normalizer(nb_inputs)


In [ ]:
train(scenario, policy, normalizer, hp)

In [ ]:
sorted_array

In [ ]:
sorted_array.flatten().shape

In [ ]:
a = np.random.rand(10,4).flatten()

In [ ]:
a.resize(50,refcheck=False)

In [ ]:
a.shape